# TensorRT Engine Report Card

Use this Jupyter worksheet to get a quick overview of the structure and characteristics of a TensorRT Engine plan.

In [1]:

# Build Engine with trex first 
# ! python thirdparty/trex/utils/process_engine.py resnet18.onnx trex_outputs int8 shapes=x:32x3x224x224

## Load JSON Files

In [1]:
import IPython
from ipywidgets import widgets
from trex import *

# Choose an engine file to load.
engine_name = "trex_outputs/resnet18.onnx.engine"

set_wide_display()

In [2]:
plan = EnginePlan(f"{engine_name}.graph.json", f"{engine_name}.profile.json", f"{engine_name}.profile.metadata.json")

### Render Graph

In [3]:
report_card_draw_plan_graph_extended(plan, engine_name)

Output()

<html><div style="text-align:center;background:#76b900;padding:20px;color:#ffffff;font-size:2em;">Plan Summary</div></html>

In [4]:
# "Average time" refers to the sum of the layer latencies, when profiling layers separately
# "Latency" refers to the [min, max, mean, median, 99% percentile] of the engine latency measurements, when timing the engine w/o profiling layers.
plan.summary()

Model:
	Inputs: x: [1, 3, 224, 224]xFP32 NCHW
	Outputs: outputs: [1, 1000]xFP32 NCHW
	Average time: 1.070 ms
	Layers: 43
	Weights: 32.6 MB
	Activations: 60.5 MB
Device Properties:
	Selected Device: NVIDIA GeForce RTX 3060
	Compute Capability: 8.6
	SMs: 28.0
	Device Global Memory: 12044 MiB
	Shared Memory per SM: 100 KiB
	Memory Bus Width: 192.0
	Application Compute Clock Rate: 1.837 GHz
	Application Memory Clock Rate: 7.501 GHz
Builder Configuration:
Performance Summary:
	Throughput: 1040.2
	Latency: [0.951294, 0.989258, 0.958596, 0.958496, 0.96051, 0.960693, 0.96167]
	Enqueue Time: [0.012207, 0.0568848, 0.0143843, 0.013916, 0.015625, 0.0163574, 0.025238]
	H2D Latency: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
	GPU Compute Time: [0.951294, 0.989258, 0.958596, 0.958496, 0.96051, 0.960693, 0.96167]
	D2H Latency: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
	Total Host Walltime: 3.0023
	Total GPU Compute Time: 2.99369


In [ ]:
layer_latency_sunburst(plan.df, "Layers Latencies (%)")

In [ ]:
report_card_table_view(plan)

### Timings

In [6]:
plot_engine_timings(timing_json_file= f"{engine_name}.timing.json")

## Performance

In [7]:
report_card_perf_overview_widget(plan);

Dropdown(description='Diagram:', options=('Latency per layer (%)', 'Latency per layer (ms)', 'Layer latency di…

## Memory Footprint

In [8]:
report_card_memory_footprint_widget(plan);

Dropdown(description='Diagram:', options=('Weights footprint per layer', 'Activation footprint per layer', 'To…

<html><div style="text-align:center;background:#76b900;padding:20px;color:#ffffff;font-size:2em;">Convolutions</div></html>

In [10]:
convs = plan.get_layers_by_type('Convolution')
report_card_convolutions_overview_widget(convs)

Dropdown(description='Diagram:', options=('Latency per convolution (color = precision)', 'Latency per convolut…

## Tactics

In [12]:
latency_vs_prec_per_conv = partial(
    plotly_bar2,
    convs,
    values_col='latency.pct_time',
    names_col='Name',
    color='tactic')

latency_vs_prec_per_conv("Latency per Layer (color=Tactics)")

tactic_cnt = group_count(plan.df, 'tactic')
display_df(tactic_cnt)

## Experimental 

The data below are based on partial information and are provided here just for exploration and fun.

Examples of simplifying assumptions:
* Convolutions and matrix-multiplications are implemented using implicit-gemm. In practice, various algorithms might be used, which will affect the arithmetic-intensity.
* Input/output activations (feature-maps) and parameters (weights and other constants) are read once, and from device global memory. In practice, it is likely that activations will be resident in the L2 cache, which has a much higher bandwidth compared to global memory. Matrix multiplication is performed using activation tiles which are read (reused) multiple times. These reads are ignored in the calculation of the arithmetic-intensity.
* Compute-efficiency and memory-efficiency are calculated by dividing the MACs or bytes by the total time per layer, instead of dividing by the time spent only on compute or only on memory access.
* Some convolutions have fued operations (e.g. SiLU activation) which are currently ignored in the calculation of the number of operations.


Performance optimization references:
* [Nvidia DL Peformance Background: Understand Performance](https://docs.nvidia.com/deeplearning/performance/dl-performance-gpu-background/index.html#understand-perf)
* [Nvidia DL Peformance: GEMM](https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#math-mem)
* [CUDA Programming: Instruction Throughput](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#maximize-instruction-throughput)
* [TensorRT Developer Guide: Performance Best Practices](https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html#performance)

In [ ]:
report_card_gemm_MNK(plan);

In [ ]:
report_card_gemm_MNK_scatter(plan);

In [ ]:
report_card_efficiency_vs_latency_3d(plan);

In [ ]:
report_card_perf_scatter(plan);

<html><div style="text-align:center;background:#76b900;padding:20px;color:#ffffff;font-size:2em;">Layer Lint Utility</div></html>

Linting functions perform static analysis of the plan to flag possible performance hazards.<br>
See TensorRT's [Performance Best Prcatices](https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html#performance) for more information.

<b>The linting is in an early experimental stage and may be imcomplete or erronous.<b>

## Convolution Lint

Ideally all Float16 and INT8 convolutions are accelerated on Tensor Cores, so `ConvLinter` uses heuristics on the kernel name to determine if the kernel is accelerated on TCs.  This method is probably incorrect is some cases and needs more investment.

In [ ]:
display_df(ConvLinter(plan).lint())

## Reformat Lint

Reformat layers copy their input to their output while making some changes to the data.<br>
A Reformat layer may change the data layout (e.g. from NCHW to NC32HW) or perform data-type conversion (e.g. from float32 to INT8).

A Reformat layer is added by to the engine graph by TensorRT's graph-optimizer for one of several reasons, which are indicated by the `attr.origin` field.
* REFORMAT: type or layout conversion.
* SLICE: slice layer output conversion.
* CONCAT: concat layer input conversion.

Reformat layers that perform data-type conversion from float32/float16 to INT8, or vice-versa, may indicate poorly placed Q/DQ layers in a QAT network. <br>
These are Q/DQ layers which could not be fused with another layer in the engine graph and may be quite costly in latency.

In [ ]:
report_card_reformat_overview(plan)

In [ ]:
display_df(ReformatLinter(plan).lint())

## Slice Linter

Slice layers that perform data-type conversion may indicate an optimization opportunity.

In [ ]:
display_df(SliceLinter(plan).lint())

## Q/DQ Linter

Quantize/Dequenatize layers perform a copy with quantization/dequantization.<br>
Unfused Q/DQ layers ("dangling Q/DQ") are very wasteful and usually indicate poorly placement of fake-quantization in the training model.

In [ ]:
display_df(QDQLinter(plan).lint())

## Pointwise Layers

Pointwise and Elementwise layers can be fused to create larger kernels.<br>
Here you can explore how well these layers managed to fuse.

In [ ]:
report_card_pointwise_lint(plan)

<html><div style="text-align:center;background:#76b900;padding:20px;color:#ffffff;font-size:2em;">Excel Summary</div></html>

### Generate a default summary

In [ ]:
summary = ExcelSummary(plan, path="default_summary.xlsx")
summary.generate_default_summary() # 'generate_default_summary' automatically saves the file

### Generate a customized summary

In [ ]:
summary = ExcelSummary(plan, path="customized_summary.xlsx")
summary.add_dataframes({"df": plan.df})
summary.add_images({"trex_logo": "../images/trex_logo.png"})
summary.add_dataframes({"clean_df": clean_for_display(plan.df)})
summary.save() # 'save' needs to be called to propagate the changes to the file

In [ ]:
# The context manager automatically saves the file
with ExcelSummary(plan, path="customized_summary_with_manager.xlsx") as summary:
    summary.add_dataframes({"df": plan.df})
    summary.add_images({"trex_logo": "../images/trex_logo.png"})
    summary.add_dataframes({"clean_df": clean_for_display(plan.df)})
